<a href="https://colab.research.google.com/github/jfarrell8/WQU_5457/blob/main/Capstone_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install Packages

In [ ]:
# install finrl library
!pip install wrds
!pip install swig
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 10.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstal

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import requests
import json
from cvxpy import *
import gym
from gym.utils import seeding
from gym import spaces

In [ ]:
from finrl import config
from finrl import config_tickers
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from finrl.meta.data_processor import DataProcessor
from finrl.meta.data_processors.processor_yahoofinance import YahooFinanceProcessor
from stable_baselines3.common.vec_env import DummyVecEnv

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ModuleNotFoundError: No module named 'finrl'

# Data Wrangling

We decided to look at diversifying our portfolio constituents rather than specifically focusing on mining companies. We're going to look at the S&P 500 Dividend Aristocrats. This is an index of the S&P 500 constituents that have increased their dividend yearly for at least the last 25 years. As of today, 03/26/2024, there are 67 companies in this index. Note that this is not a point-in-time representation of the index. Rather, we're taking the current list of 67 constituents, and building our own index with these 67 diversified companies from a past start time.

In [ ]:
sp_da = pd.read_html('https://en.wikipedia.org/wiki/S%26P_500_Dividend_Aristocrats')[0]
tickers = sp_da['Ticker symbol'].str.replace('.', '-').tolist()

In [ ]:
sp_da.Sector.unique() # let's look at the different sectors represented in this list...there are 10 --> a decent diversification

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['Industrials', 'Health Care', 'Financials', 'Materials',
       'Consumer Staples', 'Utilities', 'Information Technology',
       'Energy', 'Real Estate', 'Consumer Discretionary'], dtype=object)

In [ ]:
start_date = "2000-01-01"
end_date = "2024-04-07"

data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)['Close']
data.head()

[*********************100%%**********************]  67 of 67 completed


Ticker,ABBV,ABT,ADM,ADP,AFL,ALB,AMCR,AOS,APD,ATO,...,SHW,SJM,SPGI,SWK,SYY,TGT,TROW,WMT,WST,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,NaN,9.037449,6.413242,24.781521,6.918052,6.173300,NaN,2.339966,17.212765,8.031765,...,4.600553,9.560846,18.910173,15.583123,10.292155,22.670912,9.412900,14.469361,5.428274,18.454960
2000-01-04,NaN,8.779240,6.346435,24.781521,6.734477,6.173300,NaN,2.306249,16.688183,7.831594,...,4.429103,9.435865,18.040510,15.244358,9.971579,21.687792,9.128165,13.927941,5.484354,18.101482
2000-01-05,NaN,8.763100,6.246230,24.543232,6.695828,6.280474,NaN,2.299506,16.786543,8.081805,...,4.443390,9.373379,17.797812,15.041101,10.088720,21.196226,9.111422,13.643704,5.461923,19.088285
2000-01-06,NaN,9.069725,6.279634,24.870880,6.831099,6.494824,NaN,2.265788,17.901270,7.956707,...,4.514828,9.560846,17.312418,14.973345,10.511902,20.193438,9.379399,13.792584,5.405844,20.075123
2000-01-07,NaN,9.166557,6.379840,25.436808,6.966367,6.387651,NaN,2.360196,19.147144,8.031765,...,4.457680,9.623338,17.474211,15.244358,10.528833,21.235544,9.312405,14.834826,5.461923,20.016190


In [ ]:
data.tail()

Ticker,ABBV,ABT,ADM,ADP,AFL,ALB,AMCR,AOS,APD,ATO,...,SHW,SJM,SPGI,SWK,SYY,TGT,TROW,WMT,WST,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2024-04-01,180.759995,112.089996,62.349998,245.309998,85.190002,129.179993,9.45,89.080002,243.419998,117.849998,...,338.160004,124.730003,427.089996,95.459999,80.066124,177.820007,120.300003,60.000000,395.940002,116.989998
2024-04-02,180.690002,112.019997,62.700001,247.660004,84.930000,126.480003,9.31,88.550003,239.839996,117.769997,...,333.959991,123.059998,426.109985,94.489998,79.927002,176.240005,118.550003,59.160000,387.899994,119.279999
2024-04-03,177.330002,111.279999,62.650002,245.130005,85.180000,128.869995,9.18,88.650002,239.600006,116.970001,...,334.290009,119.519997,426.459991,96.239998,79.449997,175.490005,118.489998,59.430000,387.149994,119.300003
2024-04-04,167.899994,110.110001,63.240002,241.490005,84.360001,121.120003,9.17,87.139999,237.110001,116.129997,...,329.859985,118.790001,428.420013,94.940002,76.550003,172.220001,116.260002,59.500000,386.119995,119.720001
2024-04-05,170.000000,111.199997,63.459999,244.850006,85.419998,123.449997,9.21,87.540001,238.679993,116.230003,...,331.850006,117.220001,431.589996,95.410004,76.669998,171.770004,116.639999,59.849998,394.029999,121.370003


In [ ]:
data.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(6103, 67)

Let's try to identify start and end dates for each of these constituents to determine if we feel like there's enough data for each

In [ ]:
def find_start_end_dates(column):
    start_date = column.first_valid_index()
    end_date = column.last_valid_index()
    return start_date, end_date

In [ ]:
price_date_range = pd.DataFrame()
for column in data.columns:
    start_date, end_date = find_start_end_dates(data[column])
    price_date_range = pd.concat([price_date_range, pd.DataFrame([[column, start_date, end_date]], columns=['Ticker', 'StartDate', 'EndDate'])], axis=0)

In [ ]:
# any tickers that start really late?
price_date_range.sort_values(by='StartDate', ascending=False)

,Ticker,StartDate,EndDate
0,KVUE,2023-05-04,2024-04-05
0,ABBV,2013-01-02,2024-04-05
0,AMCR,2012-05-15,2024-04-05
0,IBM,2000-01-03,2024-04-05
0,ITW,2000-01-03,2024-04-05
...,...,...,...
0,ESS,2000-01-03,2024-04-05
0,EXPD,2000-01-03,2024-04-05
0,FAST,2000-01-03,2024-04-05
0,FRT,2000-01-03,2024-04-05


In [ ]:
# any tickers that get dropped before the current EndDate
price_date_range.EndDate.unique() # NO!

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<DatetimeArray>
['2024-04-05 00:00:00']
Length: 1, dtype: datetime64[ns]

In [ ]:
# let's drop KVUE, ABBV, AMCR since they start later than 2000-01-03
data = data.drop(['KVUE', 'ABBV', 'AMCR'], axis=1)

In [ ]:
data.shape # should be 64 columns now

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(6103, 64)

In [ ]:
data.head()

Ticker,ABT,ADM,ADP,AFL,ALB,AOS,APD,ATO,BDX,BEN,...,SHW,SJM,SPGI,SWK,SYY,TGT,TROW,WMT,WST,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,9.037449,6.413242,24.781521,6.918052,6.173300,2.339966,17.212765,8.031765,17.597893,5.676232,...,4.600553,9.560846,18.910173,15.583123,10.292155,22.670912,9.412900,14.469361,5.428274,18.454960
2000-01-04,8.779240,6.346435,24.781521,6.734477,6.173300,2.306249,16.688183,7.831594,17.135889,5.575470,...,4.429103,9.435865,18.040510,15.244358,9.971579,21.687792,9.128165,13.927941,5.484354,18.101482
2000-01-05,8.763100,6.246230,24.543232,6.695828,6.280474,2.299506,16.786543,8.081805,17.261896,5.665035,...,4.443390,9.373379,17.797812,15.041101,10.088720,21.196226,9.111422,13.643704,5.461923,19.088285
2000-01-06,9.069725,6.279634,24.870880,6.831099,6.494824,2.265788,17.901270,7.956707,17.975891,6.012103,...,4.514828,9.560846,17.312418,14.973345,10.511902,20.193438,9.379399,13.792584,5.405844,20.075123
2000-01-07,9.166557,6.379840,25.436808,6.966367,6.387651,2.360196,19.147144,8.031765,18.983883,6.056884,...,4.457680,9.623338,17.474211,15.244358,10.528833,21.235544,9.312405,14.834826,5.461923,20.016190


In [ ]:
# let's turn prices into log returns
rets = np.log(data).diff()
rets = rets.dropna()
rets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Ticker,ABT,ADM,ADP,AFL,ALB,AOS,APD,ATO,BDX,BEN,...,SHW,SJM,SPGI,SWK,SYY,TGT,TROW,WMT,WST,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,-0.028987,-0.010472,0.000000,-0.026894,0.000000,-0.014514,-0.030950,-0.025238,-0.026604,-0.017911,...,-0.037979,-0.013158,-0.047080,-0.021979,-0.031643,-0.044333,-0.030716,-0.038136,0.010278,-0.019339
2000-01-05,-0.001840,-0.015915,-0.009662,-0.005755,0.017212,-0.002928,0.005877,0.031449,0.007326,0.015936,...,0.003221,-0.006644,-0.013544,-0.013423,0.011679,-0.022926,-0.001836,-0.020619,-0.004098,0.053081
2000-01-06,0.034392,0.005334,0.013262,0.020001,0.033560,-0.014772,0.064294,-0.015600,0.040530,0.059462,...,0.015949,0.019803,-0.027651,-0.004515,0.041090,-0.048465,0.028987,0.010853,-0.010320,0.050407
2000-01-07,0.010620,0.015831,0.022500,0.019608,-0.016639,0.040822,0.067282,0.009389,0.054559,0.007421,...,-0.012739,0.006515,0.009302,0.017938,0.001609,0.050319,-0.007168,0.072846,0.010320,-0.002940
2000-01-10,-0.007067,0.000000,0.024292,-0.061492,-0.003362,0.011364,-0.063626,-0.012540,-0.022372,0.005530,...,-0.022692,-0.013072,0.067140,-0.008929,0.003210,-0.019635,0.010734,-0.018417,0.020326,-0.014079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01,-0.013909,-0.007351,-0.017898,-0.007834,-0.019624,-0.004257,0.004736,-0.008618,-0.013835,-0.017585,...,-0.026756,-0.009098,0.003847,-0.025546,-0.007543,0.003436,-0.013376,-0.002829,0.000581,0.006431
2024-04-02,-0.000625,0.005598,0.009534,-0.003057,-0.021123,-0.005967,-0.014816,-0.000679,-0.012493,-0.012386,...,-0.012498,-0.013479,-0.002297,-0.010213,-0.001739,-0.008925,-0.014654,-0.014099,-0.020515,0.019385
2024-04-03,-0.006628,-0.000798,-0.010268,0.002939,0.018720,0.001129,-0.001001,-0.006816,0.010607,0.008032,...,0.000988,-0.029188,0.000821,0.018351,-0.005986,-0.004265,-0.000506,0.004554,-0.001935,0.000168


In [ ]:
# sanity check
data.ABT/data.ABT.shift(1) - 1 # pretty close to log returns (it shouldn't be exactly equal), so looks good!

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Date
2000-01-03         NaN
2000-01-04   -0.028571
2000-01-05   -0.001838
2000-01-06    0.034991
2000-01-07    0.010676
                ...   
2024-04-01   -0.013813
2024-04-02   -0.000624
2024-04-03   -0.006606
2024-04-04   -0.010514
2024-04-05    0.009899
Name: ABT, Length: 6103, dtype: float64

In [ ]:
# add in alternative data/technical indicators
# first need to pivot the rets df
rets_long = pd.DataFrame(rets.stack())
rets_long = rets_long.reset_index()
rets_long = rets_long.rename(columns={0: 'ret'})
rets_long.head()

,Date,Ticker,ret
0,2000-01-04,ABT,-0.028987
1,2000-01-04,ADM,-0.010472
2,2000-01-04,ADP,0.000000
3,2000-01-04,AFL,-0.026894
4,2000-01-04,ALB,0.000000


## Fundamental Factors

In [ ]:
# get Alpha Vantage api key
with open("/content/drive/My Drive/WQU_Capstone/Capstone Project/alpha_vantage_api_key.txt", 'r') as f:
  AV_token = f.read()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Using Alpha Vantage

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
def get_alpha_vantage_data(statement, ticker, token):
  url = f'https://www.alphavantage.co/query?function={statement}&symbol={ticker}&apikey={token}'
  r = requests.get(url)
  return r.json()

def get_fundamental_factors(ticker, token):
  inc_s = get_alpha_vantage_data('INCOME_STATEMENT', ticker, token)
  bal_s = get_alpha_vantage_data('BALANCE_SHEET', ticker, token)
  is_df = pd.DataFrame(inc_s['quarterlyReports'])
  bs_df = pd.DataFrame(bal_s['quarterlyReports'])
  is_df = is_df.drop('reportedCurrency', axis=1)
  bs_df = bs_df.drop('reportedCurrency', axis=1)

  df = is_df.merge(bs_df, on='fiscalDateEnding')
  df['fiscalDateEnding'] = pd.to_datetime(df['fiscalDateEnding'])
  df = df.set_index('fiscalDateEnding').sort_index()
  df = df.resample('D').ffill() # <- likely will need to go to MS
  df = df.replace('None', np.nan)
  df = df.astype('float64')

  df['tic'] = ticker
  df['grossMargin'] = df.grossProfit / df.totalRevenue - 1
  df['operatingMargin'] = df.operatingIncome / df.totalRevenue - 1
  df['ROA'] = df.netIncome / df.totalAssets
  return df[['tic', 'grossMargin', 'operatingMargin', 'ROA']]

In [ ]:
mmm = get_alpha_vantage_data('INCOME_STATEMENT', 'MMM', AV_token)
mmm

{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}

In [ ]:
# FYI the below call to Alpha Vantage only allows for 25 queries a day.

In [ ]:
ff_df = pd.DataFrame() # Date, Ticker, Factor1, Factor2, ...

for ticker in tickers[:25]:
  try:
    iff_df = get_fundamental_factors(ticker, AV_token)
    ff_df = pd.concat([ff_df, iff_df], axis=0)
    print(f'Successfully appended {ticker}')
  except Exception as e:
    print(f'THERE WAS AN ISSUE WITH {ticker}')
    print(e)

THERE WAS AN ISSUE WITH MMM
'quarterlyReports'
THERE WAS AN ISSUE WITH AOS
'quarterlyReports'
THERE WAS AN ISSUE WITH ABT
'quarterlyReports'
THERE WAS AN ISSUE WITH ABBV
'quarterlyReports'
THERE WAS AN ISSUE WITH AFL
'quarterlyReports'


KeyboardInterrupt: 

In [ ]:
ff_df # example output for two random tickers (IBM, GE)

## Bond Yield Curves

In [ ]:
# use cached version
yield_curve = pd.read_csv("/content/drive/My Drive/WQU_Capstone/Capstone Project/master_yield_curve.csv")

In [ ]:
yield_curve

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,2000-01-03,NaN,NaN,5.48,NaN,5.81,6.09,6.38,6.42,6.50,6.65,6.58,6.94,6.61
1,2000-01-04,NaN,NaN,5.43,NaN,5.75,6.00,6.30,6.34,6.40,6.56,6.49,6.84,6.53
2,2000-01-05,NaN,NaN,5.44,NaN,5.74,6.05,6.38,6.43,6.51,6.68,6.62,6.95,6.64
3,2000-01-06,NaN,NaN,5.41,NaN,5.69,6.03,6.35,6.39,6.46,6.63,6.57,6.86,6.58
4,2000-01-07,NaN,NaN,5.38,NaN,5.66,6.00,6.31,6.35,6.42,6.58,6.52,6.82,6.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999,2023-12-22,5.54,5.52,5.44,5.45,5.31,4.82,4.31,4.04,3.87,3.92,3.90,4.21,4.05
6000,2023-12-26,5.53,5.52,5.45,5.44,5.28,4.83,4.26,4.05,3.89,3.91,3.89,4.20,4.04
6001,2023-12-27,5.55,5.53,5.44,5.42,5.26,4.79,4.20,3.97,3.78,3.81,3.79,4.10,3.95
6002,2023-12-28,5.57,5.55,5.45,5.42,5.28,4.82,4.26,4.02,3.83,3.84,3.84,4.14,3.98


The below function and subsequent code was used to capture the yield curve data the first time, but we no longer need to do so since we captured it in a flat file.

In [ ]:
def get_yield_curve(year):
  "Downloads the yield curve for a given year from treasury.gov"
  url_base = f"https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/{year}/all"
  full_url = f"{url_base}?field_tdr_date_value={year}&type=daily_treasury_yield_curve&page&_format=csv"
  df = pd.read_csv(full_url)
  df.set_index("Date", inplace = True)
  return df

In [ ]:
# yield_curve = pd.DataFrame()
# years = [i for i in range(2023, 1999, -1)]
# for year in years:
#   yield_curve = pd.concat([yield_curve, get_yield_curve(year)], axis=0)

# yield_curve.index = pd.to_datetime(yield_curve.index)
# yield_curve = yield_curve.sort_index()

In [ ]:
yield_curve

In [ ]:
# yield_curve.to_csv("/content/drive/My Drive/WQU_Capstone/Capstone Project/master_yield_curve.csv")

## Merging the datasets together

In [ ]:
# TBD since we reached out limit on the Alpha Vantage data. Once we have that locally
# we can better write the code to merge everything together grouped by the ticker
# and date columns

# Data Visualization

In [ ]:
# subplots of line plots for each price

In [ ]:
# subplots of line plots for each return

In [ ]:
# analyze and potentially remove some tickers if need be

# Modeling

## MVO

In [ ]:
# get annualized expected returns and covariance matrix
rets_exp = rets.mean()*252
rets_cov = rets.cov()*252

In [ ]:
rets_exp

Ticker
ABT     0.103656
ADM     0.094657
ADP     0.094595
AFL     0.103800
ALB     0.123712
          ...   
TGT     0.083631
TROW    0.103947
WMT     0.058635
WST     0.176954
XOM     0.077785
Length: 64, dtype: float64

In [ ]:
rets_cov

Ticker,ABT,ADM,ADP,AFL,ALB,AOS,APD,ATO,BDX,BEN,...,SHW,SJM,SPGI,SWK,SYY,TGT,TROW,WMT,WST,XOM
Ticker,,,,,,,,,,,,,,,,,,,,,
ABT,0.058554,0.021667,0.024820,0.026632,0.027522,0.021962,0.025867,0.017215,0.025973,0.030791,...,0.023125,0.014584,0.027290,0.027460,0.020885,0.021193,0.033256,0.017456,0.023807,0.021225
ADM,0.021667,0.095658,0.028160,0.042958,0.044238,0.032558,0.035929,0.022716,0.021229,0.045548,...,0.027161,0.019133,0.033583,0.039233,0.027844,0.029666,0.046587,0.019659,0.021809,0.037136
ADP,0.024820,0.028160,0.063941,0.040168,0.039639,0.031583,0.034222,0.022241,0.021975,0.043476,...,0.030590,0.017239,0.036855,0.040068,0.030077,0.030829,0.047876,0.021270,0.027076,0.029432
AFL,0.026632,0.042958,0.040168,0.127785,0.055490,0.045573,0.046284,0.028776,0.023289,0.069486,...,0.040023,0.018877,0.049302,0.058354,0.039658,0.041190,0.074572,0.024781,0.030812,0.043064
ALB,0.027522,0.044238,0.039639,0.055490,0.149699,0.053256,0.056274,0.029931,0.027644,0.068890,...,0.044616,0.023256,0.048690,0.062421,0.034342,0.040768,0.070112,0.021611,0.038253,0.044410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGT,0.021193,0.029666,0.030829,0.041190,0.040768,0.036988,0.036365,0.019850,0.018799,0.052144,...,0.035424,0.019487,0.038500,0.045637,0.028274,0.108033,0.057262,0.041706,0.026918,0.026106
TROW,0.033256,0.046587,0.047876,0.074572,0.070112,0.056763,0.055364,0.032005,0.029806,0.097627,...,0.049358,0.025495,0.058487,0.068485,0.039914,0.057262,0.131907,0.033289,0.042910,0.043831
WMT,0.017456,0.019659,0.021270,0.024781,0.021611,0.020736,0.023458,0.015376,0.014791,0.029817,...,0.022892,0.014884,0.022839,0.024852,0.019435,0.041706,0.033289,0.054929,0.016903,0.017776


In [ ]:
x = Variable(rets.shape[1])

# return and risk in proper matrix form
ret = rets_exp*x
risk = quad_form(x, rets_cov)

# optimize with constraints (long-only and all weights sum to 1)
prob = Problem(Minimize(risk), [sum(x)==1, x >= 0])

prob.solve()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/dist-packages/cvxpy/expressions/expression.py:613: DeprecationWarning: 
This use of ``*`` has resulted in matrix multiplication.
Usi

0.018777564945115018

In [ ]:
weights = pd.DataFrame([x.value*100], columns=rets.columns).round(2).T.rename(columns={0:'Pct'})
weights

,Pct
Ticker,
ABT,-0.00
ADM,-0.00
ADP,-0.00
AFL,0.00
ALB,0.00
...,...
TGT,0.00
TROW,0.00
WMT,7.15


In [ ]:
# show non-zero weights
nz_wts = weights[weights.Pct > 0]
nz_wts = nz_wts.sort_values(by='Pct', ascending=False)
nz_wts

,Pct
Ticker,
ED,15.00
JNJ,12.92
MCD,8.99
HRL,7.90
SJM,7.21
WMT,7.15
BDX,6.84
CHD,4.86
KMB,3.83


In [ ]:
nz_wts.sum()

Pct    100.0
dtype: float64

## Neural Nets (MLP, RNN, CNN)

In [ ]:
class PortfolioNN:
  def __init__(self):
    # any base args needed
    pass

  def loss_function(self):
    # TBD
    pass

  def train_network(self):
    # train network
    pass

  def rebalance(self):
    # portfolio rebalance
    pass


class MLP(PortfolioNN):
  def __init__(self):
    super.__init__()
    # model architecture here
    pass

class RNN(PortfolioNN):
  def __init__(self):
    super.__init__()
    # model architecture here
    pass

class CNN(PortfolioNN):
  def __init__(self):
    super.__init__()
    # model architecture here
    pass

## Backtest Neural Network Models

## Reinforcement Learning (RL)

Use FinRL vs. gym vs. Create our Own: TBD

## Build Environment

In [ ]:
# THE BELOW USES A FINRL CONCEPT PREVIOUSLY CREATED

In [ ]:
# start simple by establishing one train period vs. walk-forward train/test periods (future effort)
train = rets_long[rets_long['Date'] < '2019-01-01']
test = rets_long[rets_long['Date'] >= '2019-01-01']

In [ ]:
class StockPortfolioEnv(gym.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step


    """
    metadata = {'render.modes': ['human']}

    def __init__(self,
                df,
                stock_dim,
                hmax, # max number of shares to trade
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,))
        # Shape = (34, 30)
        # covariance matrix + technical indicators
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False
        self.turbulence_threshold = turbulence_threshold
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]


    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()

            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")

            return self.state, self.reward, self.terminal,{}

        else:
            #print("Model actions: ",actions)
            # actions are the portfolio weight
            # normalize to sum of 1
            #if (np.array(actions) - np.array(actions).min()).sum() != 0:
            #  norm_actions = (np.array(actions) - np.array(actions).min()) / (np.array(actions) - np.array(actions).min()).sum()
            #else:
            #  norm_actions = actions
            weights = self.softmax_normalization(actions)
            #print("Normalized actions: ", weights)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            #print(self.state)
            # calcualte portfolio return
            # individual stocks' return * weight
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value
            #print("Step reward: ", self.reward)
            #self.reward = self.reward*self.reward_scaling

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state

    def render(self, mode='human'):
        return self.state

    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output


    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [ ]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

In [ ]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "transaction_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4

}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

## Use Different Agents

In [ ]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

# this can be done for other agents plus custom agents as well
# custom will be important depending upon how we try new things

In [ ]:
trained_a2c = agent.train_model(model=model_a2c,
                                tb_log_name='a2c',
                                total_timesteps=50000)

In [ ]:
trained_a2c.save('/content/trained_models/trained_a2c.zip')

### Build custom agent with embargo/purging/CPCV?

## Backtest Models

# Compare Results from RL vs. NNs vs. MVO